In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
def load_pdf_file(file_path):

    loader = PyPDFLoader(file_path)
    documents = loader.load()

    return documents

In [3]:
def load_dir_file(data):
	loader =DirectoryLoader(data,glob ="*.pdf",loader_cls = PyPDFLoader)
	documents = loader.load()
	
	return documents


In [4]:
extracted_data=load_pdf_file("/Users/mohammad/Downloads/DocChat/data2/bias_testing.pdf")

In [5]:
extracted_data

[Document(page_content='Bias Testing and Mitigation in LLM-based Code\nGeneration\nDong Huang\nUniversity of Hong Kong\ndhuang@cs.hku.hk\nJie M.Zhang\nKing’s College London\njie.zhang@kcl.ac.uk\nQingwen Bu\nShanghai Jiao Tong University\nqwbu01@sjtu.edu.cn\nJunjie Chen\nCollege of Intelligence and Computing\nTianjin University\njunjiechen@tju.edu.cn\nXiaofei Xie\nSingapore Management University\nxfxie@smu.edu.sg\nHeming Cui\nUniversity of Hong Kong\nheming@cs.hku.hk\nAbstract—As the adoption of LLMs becomes more widespread\nin software coding ecosystems, a pressing issue has emerged:\ndoes the generated code contain social bias and unfairness, such\nas those related to age, gender, and race? This issue concerns\nthe integrity, fairness, and ethical foundation of software ap-\nplications that depend on the code generated by these models\nbut are underexplored in the literature. This paper presents a\nnovel bias testing framework that is specifically designed for\ncode generation tasks. 

In [6]:
len(extracted_data)

13

In [7]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 167


In [9]:
!pip install sentence-transformers

In [10]:

! pip install huggingface-hub sentence-transformers

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
!pip install --upgrade sentence-transformers

In [14]:
embeddings = download_hugging_face_embeddings()

/Users/mohammad/Downloads/DocChat/doc_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/mohammad/Downloads/DocChat/doc_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [27]:
import os
from dotenv import load_dotenv
load_dotenv()


openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print("OpenAI API Key loaded successfully.")
else:
    print("Failed to load OpenAI API Key.")

OpenAI API Key loaded successfully.


In [28]:
import os
from dotenv import load_dotenv
load_dotenv()


pinecone_api_key = os.getenv("PINECONE_API_KEY")

if pinecone_api_key:
    print("PINECONE API Key loaded successfully.")
else:
    print("Failed to load PINECONE API Key.")

PINECONE API Key loaded successfully.


In [18]:
type(pinecone_api_key)

str

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [21]:
# First upgrade pip as suggested in the warning
!python -m pip install --upgrade pip

# Then install langchain-pinecone with a compatible pinecone-client version
!pip install "pinecone-client>=3.2.2,<4.0.0" langchain-pinecone==0.1.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
from langchain.vectorstores import Pinecone

# Create a Pinecone vector store
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

NameError: name 'index_name' is not defined

In [32]:
!pip install --upgrade pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 3.2.2
    Uninstalling pinecone-client-3.2.2:
      Successfully uninstalled pinecone-client-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.1.1 requires pinecone-client<4.0.0,>=3.2.2, but you have pinecone-client 5.0.1 which is incompatible.


In [49]:
# reconnecting the vector db after cutting the connection 

from pinecone import Pinecone
# Initialize the client
pc = Pinecone(api_key="pcsk_6hSUew_NZ1Mn8bM2QfAjhCZ7twzqB2bXZKSQNd2jnqeF56XSeR2GL5DeJ7m7CZePRbGZvo")

# Connect to an existing index
index_name = "medicalbot"  # Change from "example-index" to "medicalbot"
try:
    index = pc.Index(index_name)
    print(f"Connected to index: {index_name}")
except Exception as e:
    print(f"Error connecting to index: {e}")

Connected to index: medicalbot


In [43]:
# Load Existing index
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [44]:
docsearch

In [45]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3}) 

In [46]:
retriever

VectorStoreRetriever(tags=['Pinecone', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7feb78d5d580>, search_kwargs={'k': 3})

In [50]:
retrieved_docs = retriever.invoke("What is bias in Quantum?")

In [51]:
retrieved_docs

[Document(page_content='A−i. The function Func is defined as biased for Ai if, for\ntwo different values of Ai, say v1 and v2, the output of the\nfunction changes, while all other parameters in A−i are held\nconstant. Mathematically, this is represented as:\nassert Func(A−i, Ai = v1) =Func(A−i, Ai = v2)\nIn this equation, Func (A−i, Ai = v1) and Func (A−i, Ai =\nv2) are the outputs of the function Func when Ai takes the\nvalues v1 and v2 respectively. Code bias exists if the outputs', metadata={'page': 2.0, 'source': '/Users/mohammad/Downloads/DocChat/data2/bias_testing.pdf'}),
 Document(page_content='educational-num, marital-status\nrelationship, race, gender, occupation\nEmployee [69]\nEducation, JoiningYear, PaymentTier\nAge, Gender, Everbenched, LeaveOrNot\nExperienceInCurrentDomain, City (region)\nHealth Insurance [70] age, sex (gender) , bmi, children\nsmoker, region, charges\nC. Definition of Code Bias\nInspired by the fairness definition of demographic parity\n(i.e., the outcom

In [55]:
!pip install --upgrade langchain langchain-pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain_core-0.3.28-py3-none-any.whl.metadata (6.3 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
Using cached langchain_core-0.3.28-py3-none-any.whl (411 kB)
Using cached pydantic-2.10.4-py3-none-any.whl (431 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.14.5
    Uninstalling pydantic_core-2.14.5:
      Successfully uninstalled pydantic_core-2.14.5
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.2
    Uninstalling pydantic-2.5.2:
      Successfully uninstalled pydantic-2.5.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
  Attempting uninstall: langchain-pinecone
    Found existing installation: langchain-pinecone 0.1.1
    Uninstalling langchain-pinecone-0.1

In [57]:
from langchain.vectorstores import Pinecone

# Create a Pinecone vector store
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
!pip uninstall -y langchain-openai langchain


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: langchain 0.3.13
Uninstalling langchain-0.3.13:
  Successfully uninstalled langchain-0.3.13


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [104]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [105]:
!pip install langchain-openai

  Using cached langchain_openai-0.2.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
Using cached langchain_openai-0.2.14-py3-none-any.whl (50 kB)
Using cached pydantic-2.10.4-py3-none-any.whl (431 kB)


In [106]:
!pip show langchain-openai

Name: langchain-openai
Version: 0.2.14
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/mohammad/Downloads/DocChat/doc_env/lib/python3.9/site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


In [122]:
!pip install langchain-openai openai

In [119]:
!pip uninstall -y langchain-openai pydantic

Found existing installation: langchain-openai 0.2.14
Uninstalling langchain-openai-0.2.14:
  Successfully uninstalled langchain-openai-0.2.14
Found existing installation: pydantic 2.10.4
Uninstalling pydantic-2.10.4:
  Successfully uninstalled pydantic-2.10.4


In [120]:
!pip install langchain-openai pydantic

  Using cached langchain_openai-0.2.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
Using cached langchain_openai-0.2.14-py3-none-any.whl (50 kB)
Using cached pydantic-2.10.4-py3-none-any.whl (431 kB)


In [125]:
!pip uninstall langchain langchain-openai openai pydantic -y

Found existing installation: langchain 0.3.13
Uninstalling langchain-0.3.13:
  Successfully uninstalled langchain-0.3.13
Found existing installation: langchain-openai 0.2.14
Uninstalling langchain-openai-0.2.14:
  Successfully uninstalled langchain-openai-0.2.14
Found existing installation: openai 1.58.1
Uninstalling openai-1.58.1:
  Successfully uninstalled openai-1.58.1
Found existing installation: pydantic 2.10.4
Uninstalling pydantic-2.10.4:
  Successfully uninstalled pydantic-2.10.4


In [126]:
!pip install "langchain<0.1.0" openai==0.28.1 pydantic==1.10.13

  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to pr

In [128]:
from langchain.llms import OpenAI

# Initialize the LLM
llm = OpenAI(
    temperature=0.4,
    max_tokens=500)

In [129]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [130]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [131]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [136]:
response = rag_chain.invoke({"input": "what is bias testing?"})
print(response["answer"])

Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mohammad/Downloads/DocChat/doc_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/yb/7l3gtx8n5fb22kdxg9wd88kh0000gn/T/ipykernel_7325/1284054476.py", line 1, in <module>
    response = rag_chain.invoke({"input": "what is bias testing?"})
  File "/Users/mohammad/Downloads/DocChat/doc_env/lib/python3.9/site-packages/langchain_core/runnables/base.py", line 4525, in invoke
  File "/Users/mohammad/Downloads/DocChat/doc_env/lib/python3.9/site-packages/langchain_core/runnables/base.py", line 2499, in invoke
    await sequence.ainvoke(1)
  File "/Users/mohammad/Downloads/DocChat/doc_env/lib/python3.9/site-packages/langchain_core/runnables/passthrough.py", line 469, in invoke
    # start map output stream
  File "/Users/mohammad/Downloads/DocChat/doc_env/lib/python3.9/site-packages/langchain_core/runnables/base.py", line 1626, in _cal